### 아래 주석을 통해 샘플링 진행했습니다.

In [ ]:
# # clicked == 1 데이터
# import pandas as pd

# # 데이터 로드
# all_train = pd.read_parquet("./train.parquet", engine="pyarrow")
# test = pd.read_parquet("./test.parquet", engine="pyarrow").drop(columns=['ID'])

# # print("Train shape:", all_train.shape)
# print("Test shape:", test.shape)


# clicked_1 = all_train[all_train['clicked'] == 1]

# # clicked == 0 데이터에서 동일 개수x2 만큼 무작위 추출 (다운 샘플링)
# clicked_0 = all_train[all_train['clicked'] == 0].sample(n=len(clicked_1)*1, random_state=42)

# # 두 데이터프레임 합치기    
# train = pd.concat([clicked_1, clicked_0], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
# train.to_csv("train.csv", index=False)

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import average_precision_score
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from tqdm import tqdm
import joblib

# =========================================================
# 데이터 로드 및 전처리
# =========================================================
train = pd.read_csv("./train.csv")

train['gender'].fillna(2, inplace=True)
train['age_group'].fillna(1, inplace=True)
train.drop(columns=["l_feat_17"], inplace=True)
train.fillna(0, inplace=True)

cat_cols = ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour']
train[cat_cols] = train[cat_cols].fillna("UNK")
train = pd.get_dummies(train, columns=cat_cols)

feature_cols = [c for c in train.columns if c not in ["ID", "clicked", "seq", "seq_list"]]
target_col = "clicked"

X = train[feature_cols]
y = train[target_col].values

# =========================================================
# 커스텀 메트릭
# =========================================================
def weighted_logloss(y_true, y_pred):
    eps = 1e-15
    y_pred = np.clip(y_pred, eps, 1 - eps)
    pos_frac = y_true.mean()
    neg_frac = 1 - pos_frac
    w1 = 0.5 / pos_frac
    w0 = 0.5 / neg_frac
    return -np.mean(w1 * y_true * np.log(y_pred) + w0 * (1 - y_true) * np.log(1 - y_pred))

def evaluate_score(y_true, y_pred):
    ap = average_precision_score(y_true, y_pred)
    wll = weighted_logloss(y_true, y_pred)
    return ap, wll, 0.5 * ap + 0.5 * (1 / (1 + wll))

# =========================================================
# XGBoost 파라미터
# =========================================================
params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "tree_method": "gpu_hist",
    "learning_rate": 0.010518720765916168,
    "max_depth": 10,
    "subsample": 0.883115119152337,
    "colsample_bytree": 0.6134018590653787,
    "lambda": 0.01941490269309615,
    "alpha": 0.6732668631963976,
    "min_child_weight": 6.9674958503622655,
    "seed": 42
}

# =========================================================
# Stratified KFold 학습
# =========================================================
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
oof_preds = np.zeros(len(train))
fold_scores = []

for fold, (tr_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f"\n🚀 Fold {fold+1}/{kf.n_splits}")

    dtrain = xgb.DMatrix(X.iloc[tr_idx], label=y[tr_idx])
    dvalid = xgb.DMatrix(X.iloc[val_idx], label=y[val_idx])

    model = xgb.train(
        params,
        dtrain,
        num_boost_round=1500,
        evals=[(dvalid, "valid")],
        early_stopping_rounds=50,
        verbose_eval=False
     )

    val_preds = model.predict(dvalid)
    oof_preds[val_idx] = val_preds

    ap, wll, score = evaluate_score(y[val_idx], val_preds)
    print(f"✅ Fold {fold+1}: AP={ap:.5f}, WLL={wll:.5f}, Score={score:.5f}")
    fold_scores.append(score)

print("\n=======================================")
print(f"🎯 평균 Score: {np.mean(fold_scores):.5f}")
print("=======================================")

# =========================================================
# 전체 학습 및 모델 저장
# =========================================================
dtrain_full = xgb.DMatrix(X, label=y)
final_model = xgb.train(params, dtrain_full, num_boost_round=1500)
final_model.save_model("xgb_model.json")

joblib.dump(feature_cols, "xgb_feature_cols_dev.pkl")
print("✅ 모델 및 feature 저장 완료: model.json, feature_cols.pkl")


C:\Users\user\AppData\Local\Temp\ipykernel_23816\2695643220.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['gender'].fillna(2, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_23816\2695643220.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe


🚀 Fold 1/10
✅ Fold 1: AP=0.61008, WLL=0.64514, Score=0.60897

🚀 Fold 2/10
✅ Fold 2: AP=0.61203, WLL=0.64414, Score=0.61012

🚀 Fold 3/10
✅ Fold 3: AP=0.61428, WLL=0.64397, Score=0.61128

🚀 Fold 4/10
✅ Fold 4: AP=0.61549, WLL=0.64059, Score=0.61251

🚀 Fold 5/10
✅ Fold 5: AP=0.61385, WLL=0.64308, Score=0.61123

🚀 Fold 6/10
✅ Fold 6: AP=0.61197, WLL=0.64345, Score=0.61022

🚀 Fold 7/10
✅ Fold 7: AP=0.61153, WLL=0.64399, Score=0.60990

🚀 Fold 8/10
✅ Fold 8: AP=0.61531, WLL=0.64164, Score=0.61223

🚀 Fold 9/10
✅ Fold 9: AP=0.61430, WLL=0.64222, Score=0.61162

🚀 Fold 10/10
✅ Fold 10: AP=0.61159, WLL=0.64490, Score=0.60977

🎯 평균 Score: 0.61079
✅ 모델 및 feature 저장 완료: model.json, feature_cols.pkl
